# Transfer Learning with MobileNetV3

<a name='1'></a>
## 1 - Packages

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as tfl
import tensorflow_addons as tfa

from train_utils.augmenters import data_augmenter, data_augmenter_fruit360
from train_utils.callbacks import LossHistory, LRCallBack
from train_utils.utils import is_in
from train_utils.losses import crossentropy_loss
from train_utils.dataset import food_dataset
from train_utils.lite_accuracy import evaluate_model
from train_utils.metadata_writer_for_image_classifier import generate_metadata

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

quantize_model = tfmot.quantization.keras.quantize_model

Num GPUs Available:  1


<a name='1'></a>
## 2 - Datasets

In [5]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [6]:
train_dataset, cv_dataset = food_dataset(BATCH_SIZE, IMG_SIZE)

from train_utils.dataset import fruit360_classes, all_class_names

print(
    tf.data.experimental.cardinality(train_dataset),
    train_dataset,
    sep = '\n'
)

Found 195823 files belonging to 296 classes.
Using 193865 files for training.
Found 195823 files belonging to 296 classes.
Using 1958 files for validation.
Found 89089 files belonging to 96 classes.
tf.Tensor(6059, shape=(), dtype=int64)
<BatchDataset shapes: ((None, 224, 224, 3), (None, 296)), types: (tf.float32, tf.float32)>


<a name='2'></a>
## 3 - Preprocess and Augment Training Data
(https://www.tensorflow.org/tutorials/images/data_augmentation).

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=128, reshuffle_each_iteration=True)

<a name='3'></a>
## 4 - Using MobileNetV3


In [8]:
IMG_SHAPE = IMG_SIZE + (3,)

mobile_v3_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
)

Note the last 2 layers here. They are the so called top layers, and they are responsible of the classification in the model

In [9]:
print("Number of layers in the base model: ", len(mobile_v3_model.layers))

Number of layers in the base model:  269


In [10]:
image_batch, label_batch = next(iter(train_dataset))
print(image_batch.shape, label_batch.shape)
NUM_CLASSES = label_batch.shape[-1]

(32, 224, 224, 3) (32, 296)


## Hyper-Parameters

In [11]:
## 0 to skip
FIXED_LAYERS_V3 = 256    # [0, 276]
DROPOUT_BEFORE = 0       # [0, 0.5]
TOP_DENSE_LAYER = 0    # (300, 1280) U {0}
DROPOUT_AFTER = 0        # [0, 0.5]
L1, L2 = 0, 0            # [1e-9, 1e-4] U {0}
GRADIENT_CLIP = 100       # [5, 100]

LR = (-4, -6)            # [-5.5, -2.5]
EPOCHS = 10              # [0, 30]

In [12]:
class CustomModel(tf.keras.Model):
    food_augmenter = data_augmenter()
    fruit_augmenter = data_augmenter_fruit360()
    fruit_indices = tf.constant(
        [all_class_names.index(fruit) for fruit in fruit360_classes]
    )
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) 

    def train_step(self, data):
        x, y = data
        class_indices = tf.math.argmax(y, -1)
        isfr = is_in(class_indices, self.fruit_indices)
        isfr = tf.expand_dims(tf.expand_dims(tf.expand_dims(isfr, -1), -1), -1)
        x = tf.where(
            isfr,
            self.fruit_augmenter(x),
            self.food_augmenter(x),
        )
        return super().train_step((x, y))


In [34]:
input_shape = IMG_SIZE + (3,)

base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
)
base_model.trainable = True
fine_tune_at = FIXED_LAYERS_V3
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

def food_model():
    reg = tf.keras.regularizers.L1L2(L1, L2)
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs)
    x = tfl.GlobalAveragePooling2D()(x)
    if DROPOUT_BEFORE:
        x = tf.keras.layers.Dropout(DROPOUT_BEFORE)(x)
    if TOP_DENSE_LAYER:
        x = tfl.Dense(
            TOP_DENSE_LAYER,
            activation='relu', 
            kernel_regularizer=reg)(x)
    if DROPOUT_AFTER:
        x = tf.keras.layers.Dropout(DROPOUT_AFTER)(x)
    outputs = tfl.Dense(
        NUM_CLASSES, 
        activation='softmax', 
        kernel_regularizer=reg)(x)
    
    model = CustomModel(
        inputs, 
        outputs
    )
    return model

### Estimate optimal learning rate

In [35]:
optimizer = tf.keras.optimizers.Adam(clipvalue=GRADIENT_CLIP)

metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='top1'),
    tf.keras.metrics.TopKCategoricalAccuracy(k=4, name='top4'),
]
model2 = food_model()
model2.compile(
    loss=crossentropy_loss,
    optimizer=optimizer,
    metrics=metrics,
)
model2.summary()

Model: "custom_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
MobilenetV3large (Functional (None, 1, 1, 1280)        4226432   
_________________________________________________________________
global_average_pooling2d_14  (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 296)               379176    
Total params: 4,605,608
Trainable params: 1,764,776
Non-trainable params: 2,840,832
_________________________________________________________________


# Train

In [ ]:
tqdm_callback = tfa.callbacks.TQDMProgressBar(
    metrics_separator=' ; ',
    epoch_bar_format='{n_fmt}/{total_fmt} ; ETA: {remaining}s {bar} {desc}',
    metrics_format='{name}: {value:0.4f}',
    update_per_second=1,
)
lr_callback = LRCallBack(epochs=EPOCHS, l_r=LR)

history = model2.fit(
    train_dataset,
    validation_data=cv_dataset,
    epochs=lr_callback.epochs,
    verbose=0,
    callbacks=[tqdm_callback, lr_callback],
)

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(lr_callback.batch_losses)
plt.ylabel('Loss')
plt.xlabel('Batches')
plt.title('Training Loss')

Plot the training and validation accuracy:

In [ ]:
acc = history.history['top1']
val_acc = history.history['val_top1']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 12))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,0.01])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Save the model

In [36]:
model2.save('models\\main.h5')
model2.save_weights('models\\main_weights.h5')
with open("models\\main_classes.txt", 'w') as file:
    for class_name in all_class_names:
        file.write(class_name + '\n')

## Fine Tune Quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model2)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

In [ ]:
open("models\\mobilenet_v3_large_food_classifier_op.tflite", "wb").write(tflite_model)
generate_metadata()

## Check Network Predictions

In [ ]:
image_batch, label_batch = next(iter(cv_dataset))
plt.figure(figsize=(25, 25))
for item in range(9):
    ax = plt.subplot(3, 3, item + 1)
    plt.imshow(image_batch[item].numpy().astype("uint8"))
    probs = model2(image_batch)[item]
    label = all_class_names[tf.math.argmax(label_batch[item])]
    pred = all_class_names[tf.math.argmax(probs)]
    plt.title(label + '-' + pred)
    plt.axis("off")

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

test_dataset = image_dataset_from_directory(
    'datasets\\test',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical',
)
test_labels = test_dataset.class_names


In [ ]:
image_batch, label_batch = next(iter(test_dataset))
plt.figure(figsize=(25, 25))
for item in range(1):
    ax = plt.subplot(3, 3, item + 1)
    plt.imshow(image_batch[item].numpy().astype("uint8"))
    probs = model2(image_batch)[item]
    print(image_batch[item])
    label = test_labels[tf.math.argmax(label_batch[item])]
    pred = all_class_names[tf.math.argmax(probs)]
    plt.title(label + '-' + pred)
    plt.axis("off")